In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
        # df["pred"] = df[sp] >= t_frac
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [6]:
alg_list = ["fraudar", "rsd", "rev2"][:3]
data_list = ["alpha", "otc", "amazon", "epinions"][:3]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [7]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}

../res/ilp_attack/rev2-otc/100.0-0.6.pkl
../res/ilp_attack/rev2-otc/100.0-0.8.pkl


In [ ]:
# attack_res[("alpha", "fraudar", 400.0, 0.0)].to_csv("ar.csv")

In [19]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=th_dict[a]))
    for d, a, b, f in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha  fraudar 100.0 0.0 precision  0.357513  0.370968  0.380682  0.364198
                         recall     0.907895  0.851852  0.893333  0.855072
                         f1         0.513011  0.516854  0.533865  0.510823
                     0.2 precision  0.350785  0.370968  0.380682  0.335526
                         recall     0.761364  0.741935  0.770115  0.629630
...                                      ...       ...       ...       ...
amazon rev2    500.0 0.6 recall     0.142857  0.207120  0.275862  0.329356
                         f1         0.227692  0.293578  0.334944  0.366048
                     0.8 precision  0.560606  0.503937  0.426230  0.411940
                         recall     0.115987  0.173442  0.237986  0.288100
                         f1         0.192208  0.258065  0.305433  0.339066

[669 rows x 4 columns]

In [9]:
# pd.concat([df_met.loc[("alpha", "rev2", slice(None), slice(None))], df_met.loc[("alpha", "fraudar", slice(None), slice(None))].mean(axis=1)], axis=1)
# df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
# df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[("alpha", "rev2", slice(None), "f1")].droplevel(0, axis=1)

In [20]:
def get_alg_data_met(df_met, alg="rev2", data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[(data, alg, slice(None), met)].droplevel(0, axis=1).droplevel([0, 1, 3], axis=0)
    rdf.index = [f"baseline-{i}" for i in rdf.index]
    return rdf

In [52]:
get_alg_data_met(df_met, alg="rev2", data="amazon", met="f1")

budget,100.0,200.0,300.0,400.0,500.0
baseline-0.0,0.490046,0.490709,0.490709,0.490709,0.490709
baseline-0.2,0.462302,0.450966,0.433928,0.418390,0.404135
baseline-0.4,0.445131,0.416957,0.390990,0.367492,0.347033
baseline-0.6,0.426932,0.388291,0.355790,0.329003,0.305565
baseline-0.8,0.410979,0.364386,0.326683,0.298552,0.273693
